In [ ]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
%env CUDA_LAUNCH_BLOCKING=1

# Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [ ]:
import gym
from surface_seg.envs.mcs_env import MCSEnv
from surface_seg.utils.callback import Callback
import gym.wrappers
import numpy as np
import tensorforce 
import copy
import tensorflow as tf

In [1]:
timesteps = 400

In [ ]:
def setup_env(recording=True):
    
    # Set up gym
    MCS_gym = MCSEnv(observation_fingerprints=True, 
                     observation_forces=True,
                    permute_seed=42)
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         "./vid_trpo", 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id)%20==0) #every 50, starting at 51
    
    #Convert gym to tensorforce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=timesteps,
                                         visualize=False)
    
    return env

# Set up the gym and agent in tensorforce

In [ ]:
from tensorforce.agents import Agent

agent = Agent.create(
    agent='trpo', 
    environment=setup_env(), 
    batch_size=1, 
    learning_rate=1e-3,
    memory = 40000,
    max_episode_timesteps = timesteps,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=1.0, decay_steps=10000, decay_rate=0.5
    ),
    recorder = dict(
        directory = './recorder', frequency=1), #required for recording states and actions
    summarizer = dict(
        directory = 'tb', labels='all', frequency=1, #Tensorboard summarizer
    )
)
    

agent_spec = agent.spec

In [ ]:
from tensorforce.execution import Runner
from surface_seg.utils.callback import Callback

callback = Callback('./result_trpo').episode_finish

runner = Runner(
    agent=agent_spec,
    environment=setup_env(recording=True),
    max_episode_timesteps=timesteps,
)

runner.run(num_episodes=5000, callback=callback, callback_episode_frequency=1)
# runner.run(num_episodes=100, evaluation=True)
# runner.close()

# Run the DRL method in parallel (multiple environments)

In [ ]:
# from tensorforce.execution import Runner

# runner = Runner(
#     agent=agent_spec,
#     environments=[setup_env(), setup_env()],
#     num_parallel=2,
#     max_episode_timesteps=400,
# )

# runner.run(num_episodes=1000)
# runner.run(num_episodes=100, evaluation=True)
# runner.close()